<a href="https://colab.research.google.com/github/Manish1176/RAG_with_HuggingFace/blob/main/Medical_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Medical Chatbot**

**Steps to Build the Medical Chatbot using LangChain & Hugging Face**

**1. Set Up the Environment**

Install required libraries:

!pip install langchain langchain_community langchain_huggingface transformers sentence-transformers faiss-cpu pypdf

**2. Initialize the Streamlit App**

Create a Streamlit title and input field for user queries.

Add a sidebar notification indicating the medical book is loaded.

**3. Load and Process the Medical PDF**

Use PyPDFLoader to read the medical book.

Split the text into manageable chunks using RecursiveCharacterTextSplitter.

Store these document chunks for retrieval.

**4. Generate Text Embeddings**

Use HuggingFaceEmbeddings (sentence-transformers/all-MiniLM-L6-v2) to convert text into embeddings.

Store these embeddings in a FAISS vector database for efficient retrieval.

**5. Set Up the Hugging Face Model**

Load the Flan-T5 model and tokenizer from Hugging Face (google/flan-t5-base).

Configure a text-generation pipeline with parameters like max_length, temperature, and top_p.

**6. Create the Retrieval-Augmented Generation (RAG) Chain**

Define a prompt template to format input for the model.

Implement a retrieval system using FAISS to find the most relevant chunks of text based on user questions.

Construct a LangChain processing pipeline that integrates retrieval and LLM response generation.

**7. Define the Response Generation Function**

Retrieve the most relevant document chunks.

Format them into a prompt and pass it through the Flan-T5 model.

Truncate context if necessary to fit within model token limits.

Return the model’s generated answer.

**8. Build the Streamlit Interface**

Create a text input field for user queries.

On submission, retrieve and generate a response.

Display the final answer in the Streamlit app.

**9. Run the Application**

Start the Streamlit server using:

streamlit run app.py

-- Ask medical-related questions and receive AI-generated answers.

In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [2]:
!wget -q -O - ipv4.icanhazip.com

35.240.162.104


In [3]:
!pip install langchain langchain_community langchain_huggingface transformers sentence-transformers faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
%%writefile app.py
import streamlit as st
import warnings
import torch
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Suppress warnings
warnings.filterwarnings("ignore")

# Streamlit App Title
st.title("📚 Medical Chatbot")
st.write("Ask me questions about the medical book!")

# ✅ Load and Process PDF
def load_and_process_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    docs = text_splitter.split_documents(data)
    return docs

pdf_path = "/content/Medical_Book.pdf"  # Ensure correct path
st.sidebar.write("✅ Loaded Medical Book PDF")
docs = load_and_process_pdf(pdf_path)

# ✅ Load Hugging Face Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ Store embeddings in FAISS
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

# ✅ Load T5 Model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map="auto")

# ✅ Setup Hugging Face Pipeline
text_generation_pipeline = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=300,  # Slightly increased response length
    min_length=50,
    num_return_sequences=1,
    temperature=0.7,
    do_sample=True,
    top_p=0.95,
    repetition_penalty=1.2
)


llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# ✅ Define Prompt Template
prompt_template = """
You are a knowledgeable AI trained on medical information. Read the following context carefully and provide a clear, concise, and well-structured answer.

Context:
{context}

Question:
{question}

Provide an answer in full sentences, ensuring clarity and completeness.
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

# ✅ Function to Truncate Text
def truncate_text(text, max_tokens=400):
    tokens = text.split()
    return " ".join(tokens[:max_tokens])

# ✅ Create LLM Chain
llm_chain = prompt | llm | StrOutputParser()
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

# ✅ Function to Generate Responses
def generate_response(question):
    retrieved_contexts = retriever.invoke(question)
    combined_context = "\n\n".join([truncate_text(doc.page_content, max_tokens=500) for doc in retrieved_contexts])  # Increase tokens

    if not combined_context.strip():
        return "I couldn't find relevant information in the provided context."

    formatted_prompt = prompt.format(context=combined_context, question=question)
    response = llm.invoke(formatted_prompt)

    return response.strip()


# ✅ Streamlit User Input
user_question = st.text_input("💬 Ask a question:")
if user_question:
    with st.spinner("Thinking..."):
        answer = generate_response(user_question)
    st.write("💡 Answer:", answer)


Writing app.py


In [5]:
! streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸

⠼⠴⠦⠧⠇⠏⠋
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.240.162.104:8501

⠙⠹⠸Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴your url is: https://spotty-chairs-kneel.loca.lt
2025-03-31 06:52:34.774454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743403955.085089    1265 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743403955.178582    1265 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-31 06:52:35.904666: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorF